#### Objective Gendered Corefrence Resolution


Questions-

1. How POS is related to Coreference relations?
2. Different parmeters and it's impact on Corefrence?

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../gap/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("./gap"))

# Any results you write to the current directory are saved as output.

['constants.py', 'CONTRIBUTING.md', 'gap-development.tsv', 'gap-test.tsv', 'gap-validation.tsv', 'gap_scorer.py', 'LICENSE', 'README.md', 'sample_submission_stage_1.csv', 'test_stage_1.tsv', 'test_stage_1.tsv.zip']


In [29]:
#Getting train data from github google repo

gh_test = pd.read_csv("https://raw.githubusercontent.com/google-research-datasets/gap-coreference/master/gap-test.tsv", delimiter='\t')
gh_valid = pd.read_csv("https://raw.githubusercontent.com/google-research-datasets/gap-coreference/master/gap-validation.tsv", delimiter='\t')
train = pd.concat((gh_test, gh_valid)).rename(columns={'A': 'A_Noun', 'B': 'B_Noun'}).reset_index(drop=True)
train.shape

(2454, 11)

In [30]:
#Loading test data and output sample
test_df = pd.read_csv("./gap/test_stage_1.tsv", delimiter='\t').rename(columns={'A': 'A_Noun', 'B': 'B_Noun'})
submission = pd.read_csv('./gap/sample_submission_stage_1.csv')

In [9]:
train.to_csv(r'gaptrain.csv')

In [32]:
train['Text'][0]

"Upon their acceptance into the Kontinental Hockey League, Dehner left Finland to sign a contract in Germany with EHC M*nchen of the DEL on June 18, 2014. After capturing the German championship with the M*nchen team in 2016, he left the club and was picked up by fellow DEL side EHC Wolfsburg in July 2016. Former NHLer Gary Suter and Olympic-medalist Bob Suter are Dehner's uncles. His cousin is Minnesota Wild's alternate captain Ryan Suter."

In [31]:
#Unique pronouns in train data
train['Pronoun'].unique()

array(['His', 'him', 'He', 'his', 'her', 'She', 'he', 'she', 'Her',
       'hers'], dtype=object)

- Capitalized first letter pronouns are starting the sentence.
- Subject pronouns - he/she
- Object pronouns - him/her
- possessive - his/her/hers

In [33]:
#get set of entire data to build exploratory template
train_sample = train.head(n=15)

In [34]:
train_sample

,ID,Text,Pronoun,Pronoun-offset,A_Noun,A-offset,A-coref,B_Noun,B-offset,B-coref,URL
0,test-1,Upon their acceptance into the Kontinental Hoc...,His,383,Bob Suter,352,False,Dehner,366,True,http://en.wikipedia.org/wiki/Jeremy_Dehner
1,test-2,"Between the years 1979-1981, River won four lo...",him,430,Alonso,353,True,Alfredo Di St*fano,390,False,http://en.wikipedia.org/wiki/Norberto_Alonso
2,test-3,Though his emigration from the country has aff...,He,312,Ali Aladhadh,256,True,Saddam,295,False,http://en.wikipedia.org/wiki/Aladhadh
3,test-4,"At the trial, Pisciotta said: ``Those who have...",his,526,Alliata,377,False,Pisciotta,536,True,http://en.wikipedia.org/wiki/Gaspare_Pisciotta
4,test-5,It is about a pair of United States Navy shore...,his,406,Eddie,421,True,Rock Reilly,559,False,http://en.wikipedia.org/wiki/Chasers
5,test-6,The others were Adam Baldwin (Jayne Cobb in Fi...,her,349,Jewel Staite,281,True,Keller,310,False,http://en.wikipedia.org/wiki/Jennifer_Keller
6,test-7,"Allison Fischer (born October 19, 1988) is an ...",She,365,Allison,232,True,Grace Smythe,290,False,http://en.wikipedia.org/wiki/Allison_Fischer
7,test-8,"The monster arrives and bites Jeni's tongue, b...",her,307,Sophie,248,False,Jeni,277,True,http://en.wikipedia.org/wiki/Leprechaun:_Origins
8,test-9,"On June 4, 1973 at the Felt Forum, Madison Squ...",he,227,Malave,124,True,Greg Joiner,169,False,http://en.wikipedia.org/wiki/Edwin_Malave
9,test-10,Go Away (Lorrie Morgan song) ``Go Away'' is a ...,her,223,Cathy Majeski,78,False,Lorrie Morgan,154,True,http://en.wikipedia.org/wiki/Go_Away_(Lorrie_M...


In [35]:
#replace function
def name_replace(s, r1, r2):
    s = str(s).replace(r1,r2)
  #  for r3 in r1.split(' '):
  #      s = str(s).replace(r3,r2)
    return s

In [36]:
import string
def get_features(df):
    df['char_count'] = df['Text'].apply(len) 
    df['word_count'] = df['Text'].apply(lambda x: len(x.split()))
    df['word_density'] = df['char_count'] / (df['word_count']+1)
    df['punctuation_count'] = df['Text'].apply(lambda x: len("".join(_ for _ in x if _ in string.punctuation))) 
    df['title_word_count'] = df['Text'].apply(lambda x: len([wrd for wrd in x.split() if wrd.istitle()]))
    df['upper_case_word_count'] = df['Text'].apply(lambda x: len([wrd for wrd in x.split() if wrd.isupper()]))
    df['section_min'] = df[['Pronoun-offset', 'A-offset', 'B-offset']].min(axis=1)
    df['Pronoun-offset2'] = df['Pronoun-offset'] + df['Pronoun'].map(len)
    df['A-offset2'] = df['A-offset'] + df['A_Noun'].map(len)
    df['B-offset2'] = df['B-offset'] + df['B_Noun'].map(len)                               
    df['section_max'] = df[['Pronoun-offset2', 'A-offset2', 'B-offset2']].max(axis=1)
    df['Text'] = df.apply(lambda r: name_replace(r['Text'], r['A_Noun'], 'anoun'), axis=1)
    df['Text'] = df.apply(lambda r: name_replace(r['Text'], r['B_Noun'], 'bnoun'), axis=1)
    df['A_count'] = df.Text.str.count('anoun')
    df['B_count'] = df.Text.str.count('bnoun')
    df['A-dist'] = (df['Pronoun-offset'] - df['A-offset']).abs()
    df['B-dist'] = (df['Pronoun-offset'] - df['B-offset']).abs()
    return(df)

train = get_features(train)
test = get_features(test_df)

In [16]:
# Loading NLP libraries
import spacy
from spacy import displacy
nlp = spacy.load('en_core_web_sm')
import nltk

In [39]:
train.head()

,ID,Text,Pronoun,Pronoun-offset,A_Noun,A-offset,A-coref,B_Noun,B-offset,B-coref,...,upper_case_word_count,section_min,Pronoun-offset2,A-offset2,B-offset2,section_max,A_count,B_count,A-dist,B-dist
0,test-1,Upon their acceptance into the Kontinental Hoc...,His,383,Bob Suter,352,False,Dehner,366,True,...,4,352,386,361,372,386,1,2,31,17
1,test-2,"Between the years 1979-1981, River won four lo...",him,430,Alonso,353,True,Alfredo Di St*fano,390,False,...,0,353,433,359,408,433,2,1,77,40
2,test-3,Though his emigration from the country has aff...,He,312,Ali Aladhadh,256,True,Saddam,295,False,...,1,256,314,268,301,314,2,1,56,17
3,test-4,"At the trial, bnoun said: ``Those who have mad...",his,526,Alliata,377,False,Pisciotta,536,True,...,1,377,529,384,545,545,3,2,149,10
4,test-5,It is about a pair of United States Navy shore...,his,406,Eddie,421,True,Rock Reilly,559,False,...,1,406,409,426,570,570,2,1,15,153


In [40]:
def get_nlp_features(s, w):
    doc = nlp(str(s))
    tokens = pd.DataFrame([[token.text, token.dep_] for token in doc], columns=['text', 'dep'])
    return len(tokens[((tokens['text']==w) & (tokens['dep']=='poss'))])

train['A-poss'] = train['Text'].map(lambda x: get_nlp_features(x, 'anoun'))
train['B-poss'] = train['Text'].map(lambda x: get_nlp_features(x, 'bnoun'))
test['A-poss'] = test['Text'].map(lambda x: get_nlp_features(x, 'anoun'))
test['B-poss'] = test['Text'].map(lambda x: get_nlp_features(x, 'bnoun'))

In [41]:
train = train.rename(columns={'A-coref':'A', 'B-coref':'B'})
train['A'] = train['A'].astype(int)
train['B'] = train['B'].astype(int)
train['NEITHER'] = 1.0 - (train['A'] + train['B'])

In [48]:
from sklearn import *
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from xgboost import XGBClassifier

col = ['char_count','word_count','word_density','punctuation_count','title_word_count','upper_case_word_count','section_min', 'Pronoun-offset2', 'A_count', 'B_count', 'A-offset2', 'B-offset2', 'section_max', 'A-poss', 'B-poss', 'A-dist', 'B-dist']
x1, x2, y1, y2 = model_selection.train_test_split(train[col].fillna(-1), train[['A', 'B', 'NEITHER']], test_size=0.2, random_state=1)
x1.head()

,char_count,word_count,word_density,punctuation_count,title_word_count,upper_case_word_count,section_min,Pronoun-offset2,A_count,B_count,A-offset2,B-offset2,section_max,A-poss,B-poss,A-dist,B-dist
1699,544,93,5.787234,15,24,4,379,451,1,1,385,401,451,0,0,69,58
1790,516,80,6.370370,11,23,3,315,377,1,2,329,358,377,0,2,59,28
1665,456,78,5.772152,15,9,1,297,352,1,2,306,334,352,0,0,52,20
898,376,62,5.968254,11,13,0,262,302,3,3,265,285,302,0,1,37,18
270,444,74,5.920000,21,16,1,336,433,1,2,340,385,433,0,1,95,52


In [52]:
model = multiclass.OneVsRestClassifier(ensemble.RandomForestClassifier(max_depth = 7, n_estimators=1000, random_state=33))
# model = multiclass.OneVsRestClassifier(ensemble.ExtraTreesClassifier(n_jobs=-1, n_estimators=100, random_state=33))

# param_dist = {'objective': 'binary:logistic', 'max_depth': 1, 'n_estimators':1000, 'num_round':1000, 'eval_metric': 'logloss'}
# model = multiclass.OneVsRestClassifier(xgb.XGBClassifier(**param_dist))

model.fit(x1, y1)


y1.head()

,A,B,NEITHER
1699,1,0,0.0
1790,1,0,0.0
1665,1,0,0.0
898,1,0,0.0
270,0,1,0.0


In [54]:
model.predict_proba(x2)

array([[0.40990296, 0.38521437, 0.15524851],
       [0.55801026, 0.34676216, 0.08010698],
       [0.56544831, 0.28365129, 0.2957617 ],
       ...,
       [0.43607137, 0.38007422, 0.14192504],
       [0.7292482 , 0.20795145, 0.08734859],
       [0.65349419, 0.3029094 , 0.07773353]])

In [55]:
y2.head()

,A,B,NEITHER
120,0,0,1.0
768,0,1,0.0
521,0,0,1.0
107,1,0,0.0
1652,1,0,0.0


In [56]:
print('log_loss', metrics.log_loss(y2, model.predict_proba(x2)))

log_loss 0.8593135031612028


In [49]:
model = multiclass.OneVsRestClassifier(ensemble.RandomForestClassifier(max_depth = 7, n_estimators=1000, random_state=33))
# model = multiclass.OneVsRestClassifier(ensemble.ExtraTreesClassifier(n_jobs=-1, n_estimators=100, random_state=33))

# param_dist = {'objective': 'binary:logistic', 'max_depth': 1, 'n_estimators':1000, 'num_round':1000, 'eval_metric': 'logloss'}
# model = multiclass.OneVsRestClassifier(xgb.XGBClassifier(**param_dist))

model.fit(x1, y1)
print('log_loss', metrics.log_loss(y2, model.predict_proba(x2)))
model.fit(train[col].fillna(-1), train[['A', 'B', 'NEITHER']])
results = model.predict_proba(test[col])
test['A'] = results[:,0]
test['B'] = results[:,1]
test['NEITHER'] = results[:,2]
test[['ID', 'A', 'B', 'NEITHER']].to_csv('submission.csv', index=False)

log_loss 0.8593135031612028


In [ ]:
def my_model(x1, x2, y1, y2):
    model.fit(x1, y1)
    log_loss = metrics.log_loss(y2, model.predict_proba(x2))
    print('log_loss', log_loss)
    return log_loss

In [ ]:
from mlxtend.feature_selection import SequentialFeatureSelector as sfs

In [ ]:
# Build step forward feature selection
sfs1 = sfs(model,
           k_features=5,
           forward=True,
           floating=False,
           verbose=2,
           scoring='accuracy',
           cv=5)

# Perform SFFS
sfs1 = sfs1.fit(x1, y1)

In [ ]:
# Which features?
feat_cols = list(sfs1.k_feature_idx_)
print(feat_cols)

In [ ]:
print(col)

In [ ]:
col = ['B_count', 'A-poss', 'B-poss']
x1, x2, y1, y2 = model_selection.train_test_split(train[col].fillna(-1), train[['A', 'B', 'NEITHER']], test_size=0.2, random_state=1)

model.fit(x1, y1)
print('log_loss', metrics.log_loss(y2, model.predict_proba(x2)))
model.fit(train[col].fillna(-1), train[['A', 'B', 'NEITHER']])
results = model.predict_proba(test[col])
test['A'] = results[:,0]
test['B'] = results[:,1]
test['NEITHER'] = results[:,2]
test[['ID', 'A', 'B', 'NEITHER']].to_csv('submission.csv', index=False)

In [46]:
col = ['A-poss', 'B-poss']
x1, x2, y1, y2 = model_selection.train_test_split(train[col].fillna(-1), train[['A', 'B', 'NEITHER']], test_size=0.2, random_state=1)

#model = multiclass.OneVsRestClassifier(ensemble.ExtraTreesClassifier(n_jobs=-1, n_estimators=100, random_state=33))

param_dist = {'objective': 'binary:logistic', 'max_depth': 1, 'n_estimators':1000, 'num_round':1000, 'eval_metric': 'logloss'}
model = multiclass.OneVsRestClassifier(xgb.XGBClassifier(**param_dist))

model.fit(x1, y1)
print('log_loss', metrics.log_loss(y2, model.predict_proba(x2)))
model.fit(train[col].fillna(-1), train[['A', 'B', 'NEITHER']])
results = model.predict_proba(test[col])
test['A'] = results[:,0]
test['B'] = results[:,1]
test['NEITHER'] = results[:,2]
test[['ID', 'A', 'B', 'NEITHER']].to_csv('submission.csv', index=False)

log_loss 0.9750405067460357


In [ ]:
import pickle
# Let's pickle it for later use
train.to_pickle("train.pkl")
test.to_pickle("test.pkl")